<a href="https://colab.research.google.com/github/Elysa-G/programming-assignment-0/blob/main/Assignment_0-aws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with data that was downloaded using using the [yfinance](https://pypi.org/project/yfinance/) library. It containst daily closing prices of all symbols.

In [ ]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd

In [ ]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())


In [ ]:
#Describe your answer below.
#Answer 1: pd.read_csv reads in the data from the CSV file at the indicated URL, delimiting each data point by '|',
#          data_clean is then a subset of all that data, specfically the data points where the value for Test Issue is N,
#          and the final line first makes data_clean a list with all symbols in the NASDAQ column of the cleaned data subset.
#          It then create a set of unique symbols, stored in symbols \n"

In [ ]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

In [ ]:
!ls stock_closing_prices.parquet -l #downloaded file

-rw-r--r-- 1 root root 1011183 Jan 20 02:52 stock_closing_prices.parquet


In [ ]:
#Bonus question 1 - find on internet and write the command that will download the all data from the symbols list. 

In [ ]:
import pandas as pd
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
ds = pd.DataFrame(symbols)
ds.to_csv('bonusq.csv', sep='\t')

In [ ]:
data=pd.read_parquet('stocks.parquet')

In [ ]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock


,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday


In [ ]:
# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the adj_close on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z
# Sort the stocks by this return and find the top 20

In [1]:
import pandas as pd

!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data
data=pd.read_parquet('stocks.parquet', columns=['date','adj_close','symbol'])
data['date']=pd.to_datetime(data['date'])
symboldates=data.groupby('symbol',as_index=False).agg(date_min=pd.NamedAgg(column="date", aggfunc="min"),date_max=pd.NamedAgg(column="date", aggfunc="max"))
df=symboldates.merge(data.rename(columns={'adj_close':'start_adj_close'}), left_on=['symbol','date_min'],right_on=['symbol','date']).drop('date', axis=1)
df=df.merge(data.rename(columns={'adj_close':'end_adj_close'}),left_on=['symbol','date_max'],right_on=['symbol','date']).drop('date', axis=1)

df['num_days']= (df.date_max-df.date_min).dt.days
df['return'] = df.apply(lambda row: None if row.start_adj_close ==0 or row.num_days==0 else row.end_adj_close/row.start_adj_close/row.num_days, axis =1)
df=df.sort_values('return', ascending=False)
df=df.head(20)
df

ModuleNotFoundError: ignored

In [ ]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS